In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

## Setup

We're going to download the collected works of Nietzsche to use as our data for this class.

In [2]:
PATH='data/nietzsche/'

In [3]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

corpus length: 600893


In [4]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

Map from chars to indices and back again

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [9]:
idx = [char_indices[c] for c in text]

In [10]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [11]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## Three char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [12]:
cs=3
c1_dat = [idx[i]   for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

Our inputs

In [13]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

Our output

In [14]:
y = np.stack(c4_dat[:-2])

The first 4 inputs and outputs

In [14]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [15]:
y[:4]

array([30, 29,  1, 40])

In [16]:
x1.shape, y.shape

((200295,), (200295,))

### Create and train model

Pick a size for our hidden state

In [15]:
n_hidden = 256

The number of latent factors to create (i.e. the size of the embedding matrix)

In [16]:
n_fac = 42

In [30]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda())
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [20]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)

In [21]:
m = Char3Model(vocab_size, n_fac).cuda()

In [22]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [191]:
opt = optim.Adam(m.parameters(), 1e-2)

In [192]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       2.09627  6.52849]                                 



In [193]:
set_lrs(opt, 0.001)

In [194]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.84525  6.52312]                                 



### Test model

In [195]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [196]:
get_next('y. ')

'T'

In [197]:
get_next('ppl')

'e'

In [198]:
get_next(' th')

'e'

In [199]:
get_next('and')

' '

## Our first RNN!

### Create inputs

This is the size of our unrolled RNN.

In [63]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [64]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs-1)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [65]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs-1)]

In [66]:
xs = np.stack(c_in_dat, axis=0)

In [67]:
xs.shape

(600884, 8)

In [68]:
y = np.stack(c_out_dat)

So each column below is one series of 8 characters from the text.

In [69]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [42, 29, 30, 25, 27, 29,  1,  1],
       [29, 30, 25, 27, 29,  1,  1,  1],
       [30, 25, 27, 29,  1,  1,  1, 43],
       [25, 27, 29,  1,  1,  1, 43, 45],
       [27, 29,  1,  1,  1, 43, 45, 40],
       [29,  1,  1,  1, 43, 45, 40, 40],
       [ 1,  1,  1, 43, 45, 40, 40, 39]])

...and this is the next character after each sequence.

In [70]:
y[:cs]

array([ 1,  1, 43, 45, 40, 40, 39, 43])

### Create and train model

In [71]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [72]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [81]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [82]:
m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [83]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       2.02986  1.99268]                                



In [84]:
set_lrs(opt, 0.001)

In [85]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.73588  1.75103]                                 



In [92]:
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [93]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [94]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [95]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.81654  1.78501]                                



In [96]:
set_lrs(opt, 1e-4)

In [97]:
fit(m, md, 1, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.69008  1.69936]                                 



### Test model

In [98]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [99]:
get_next('for thos')

'e'

In [100]:
get_next('part of ')

't'

In [101]:
get_next('queens a')

'n'

## RNN with pytorch

In [108]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [109]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [110]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [111]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([8, 512, 42])

In [112]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [113]:
t = m(*V(xs)); t.size()

torch.Size([512, 85])

In [114]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.86065  1.84255]                                 
[ 1.       1.68014  1.67387]                                 
[ 2.       1.58828  1.59169]                                 
[ 3.       1.52989  1.54942]                                 



In [115]:
set_lrs(opt, 1e-4)

In [116]:
fit(m, md, 2, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.46841  1.50966]                                 
[ 1.       1.46482  1.5039 ]                                 



### Test model

In [117]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [118]:
get_next('for thos')

'e'

In [119]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [120]:
get_next_n('for thos', 40)

'for those the same the same the same the same th'

## Multi-output model

### Setup

Let's take non-overlapping sets of characters this time

In [121]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

Then create the exact same thing, offset by 1, as our labels

In [122]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [123]:
xs = np.stack(c_in_dat)
xs.shape

(75111, 8)

In [124]:
ys = np.stack(c_out_dat)
ys.shape

(75111, 8)

In [125]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [ 1,  1, 43, 45, 40, 40, 39, 43],
       [33, 38, 31,  2, 73, 61, 54, 73],
       [ 2, 44, 71, 74, 73, 61,  2, 62],
       [72,  2, 54,  2, 76, 68, 66, 54],
       [67,  9,  9, 76, 61, 54, 73,  2],
       [73, 61, 58, 67, 24,  2, 33, 72],
       [ 2, 73, 61, 58, 71, 58,  2, 67]])

In [126]:
ys[:cs,:cs]

array([[42, 29, 30, 25, 27, 29,  1,  1],
       [ 1, 43, 45, 40, 40, 39, 43, 33],
       [38, 31,  2, 73, 61, 54, 73,  2],
       [44, 71, 74, 73, 61,  2, 62, 72],
       [ 2, 54,  2, 76, 68, 66, 54, 67],
       [ 9,  9, 76, 61, 54, 73,  2, 73],
       [61, 58, 67, 24,  2, 33, 72,  2],
       [73, 61, 58, 71, 58,  2, 67, 68]])

### Create and train model

In [127]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [128]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [133]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [134]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [135]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [136]:
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [137]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       2.59241  2.40251]                                
[ 1.       2.28474  2.19859]                                
[ 2.       2.13883  2.08836]                                
[ 3.       2.04892  2.01564]                                



In [138]:
set_lrs(opt, 1e-4)

In [139]:
fit(m, md, 1, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       1.99819  2.00106]                               



### Identity init!

In [140]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [141]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [142]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       2.39428  2.21111]                                
[ 1.       2.10381  2.03275]                                
[ 2.       1.99451  1.96393]                               
[ 3.       1.93492  1.91763]                                



In [143]:
set_lrs(opt, 1e-3)

In [144]:
fit(m, md, 4, opt, nll_loss_seq)

A Jupyter Widget

[ 0.       1.84035  1.85742]                                
[ 1.       1.82896  1.84887]                                
[ 2.       1.81879  1.84281]                               
[ 3.       1.81337  1.83801]                                



## Stateful model

### Setup

In [12]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

In [13]:
PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

nietzsche.txt  trn/  val/


In [14]:
TEXT = data.Field(lower=True, tokenize=list)

In [15]:
bs=64; bptt=8; n_fac=42; n_hidden=256

In [16]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

In [17]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(963, 56, 1, 493747)

### RNN

In [18]:
class CharSeqStatefulRnn(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        self.vocab_size = vocab_size
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(1, bs, n_hidden)))

In [19]:
m = CharSeqStatefulRnn(md.nt, n_fac, 512).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [20]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.81545  1.81713]                                 
[ 1.       1.62603  1.64905]                                 
[ 2.       1.54793  1.57219]                                 
[ 3.       1.4961   1.53892]                                 



In [21]:
set_lrs(opt, 1e-4)

In [22]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.41512  1.49226]                                 
[ 1.       1.41643  1.49173]                                 
[ 2.       1.41252  1.49211]                                 
[ 3.       1.40713  1.48223]                                 



### LSTM

In [72]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(1, bs, n_hidden)), V(torch.zeros(1, bs, n_hidden)))

In [73]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512).cuda()

In [26]:
opt = optim.Adam(m.parameters(), 1e-2)

In [27]:
fit(m, md, 6, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.5438   1.54785]                                 
[ 1.       1.46619  1.49483]                                 
[ 2.       1.42827  1.47593]                                 
[ 3.       1.41521  1.46032]                                 
[ 4.       1.40095  1.46784]                                 
[ 5.       1.39246  1.46071]                                 



In [28]:
set_lrs(opt, 1e-3)

In [29]:
fit(m, md, 3, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.23228  1.37773]                                 
[ 1.       1.22259  1.36305]                                 
[ 2.       1.21478  1.35793]                                 



### Putting it all together

In [18]:
from fastai import sgdr

In [56]:
class CharSeqStatefulLSTM2(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.25)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [57]:
m = CharSeqStatefulLSTM2(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [58]:
fit(m, md, 2, lo.opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.61586  1.5911 ]                                 
[ 1.       1.53229  1.50647]                                 



In [59]:
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2)]
fit(m, md, 2**5-1, lo.opt, F.nll_loss, callbacks=cb)

A Jupyter Widget

[ 0.       1.38319  1.3851 ]                                 
[ 1.       1.43515  1.4244 ]                                 
[ 2.       1.31039  1.33648]                                 
[ 3.       1.46857  1.45478]                                 
[ 4.       1.38676  1.39508]                                 
[ 5.       1.30398  1.331  ]                                 
[ 6.       1.2355   1.30879]                                 
[ 7.       1.45768  1.45619]                                 
[ 8.       1.41471  1.42447]                                 
[ 9.       1.37281  1.40442]                                 
[ 10.        1.3315    1.36899]                              
[ 11.        1.2885    1.33399]                              
[ 12.        1.24615   1.30934]                              
[ 13.        1.20056   1.28775]                              
[ 14.        1.17688   1.27876]                              
[ 15.        1.41478   1.44928]                              
[ 16.   

### Test

In [61]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [62]:
get_next('for thos')

'e'

In [63]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [66]:
print(get_next_n('for thos', 400))

for those has. he deeply; he was a kind of very dangerous, without whenever absolutely, to be standard, heights of philosophers--following, absurdization; who should have regards
like it,
then, acknowledge harmless, like things alite do at all regard
that remains for their changes; but we not rule the power,
the hell of the wrepks oneself preferred the presumable and possible of everythiness, ultrarment, 
